In [1]:
#Name             : Anjani Kumar

### Use Case : Recommendation Based on User Purchased Behaviour

### Import libraries

In [2]:
import pandas as pd
from scipy.spatial.distance import cosine
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Training Data
train_data=pd.read_csv('train_data_anjani.csv')

In [4]:
train_data.head()

,transaction_id,item_id,item_qty,InvoiceDate,amount,customer_id
0,536365,85123A,6,12/1/2010 8:26,2.55,17850.0
1,536365,71053,6,12/1/2010 8:26,3.39,17850.0
2,536365,84406B,8,12/1/2010 8:26,2.75,17850.0
3,536365,84029G,6,12/1/2010 8:26,3.39,17850.0
4,536365,84029E,6,12/1/2010 8:26,3.39,17850.0


In [5]:
train_data.shape

(864476, 6)

In [6]:
#Submission Data Set
submit_data=pd.read_csv('submission.csv')

In [7]:
submit_data.head()

,customer_id,Target_20724,Target_20725,Target_20726,Target_20727,Target_20728,Target_20914,Target_21080,Target_21212,Target_21232,...,Target_21931,Target_21977,Target_22178,Target_22197,Target_22355,Target_84879,Target_84991,Target_85099B,Target_85099C,Target_85123A
0,12347,1,0,0,1,1,1,0,0,1,...,1,0,1,1,0,0,0,1,1,1
1,12349,1,1,1,0,1,1,1,0,0,...,1,0,0,0,0,0,1,0,0,1
2,12352,0,0,1,0,1,0,0,0,0,...,0,0,0,1,1,1,1,0,0,1
3,12357,0,0,1,0,0,1,1,1,1,...,0,0,0,0,1,1,0,1,1,0
4,12359,0,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [8]:
submit_data.shape

(1443, 22)

### Select all the columns except first from Submission Data Set

In [9]:
cols=submit_data.columns.tolist()[1:]

In [10]:
item_list=[]
for col in cols:
    val=col[7:]
    item_list.append(val)

In [11]:
my_df=train_data[train_data['item_id'].isin(item_list)]

In [12]:
my_df.shape

(47379, 6)

### Exploratory Data Analysis

In [13]:
my_df.describe()

,item_qty,amount,customer_id
count,47379.000000,47379.000000,39138.000000
mean,17.626754,2.000230,15288.745260
std,75.997418,1.518736,1670.258059
min,-9200.000000,0.000000,12347.000000
25%,2.000000,0.850000,13871.000000
50%,8.000000,1.650000,15164.000000
75%,15.000000,2.550000,16746.000000
max,4300.000000,26.040000,18287.000000


In [14]:
submit_data.describe()

,customer_id,Target_20724,Target_20725,Target_20726,Target_20727,Target_20728,Target_20914,Target_21080,Target_21212,Target_21232,...,Target_21931,Target_21977,Target_22178,Target_22197,Target_22355,Target_84879,Target_84991,Target_85099B,Target_85099C,Target_85123A
count,1443.000000,1443.000000,1443.000000,1443.000000,1443.000000,1443.000000,1443.000000,1443.000000,1443.000000,1443.000000,...,1443.000000,1443.00000,1443.000000,1443.00000,1443.000000,1443.000000,1443.000000,1443.000000,1443.00000,1443.000000
mean,15300.927235,0.503812,0.498267,0.498960,0.501733,0.511435,0.481635,0.504505,0.521830,0.490644,...,0.495495,0.49203,0.491337,0.50797,0.507277,0.483714,0.501040,0.511435,0.49203,0.477477
std,1700.941360,0.500159,0.500170,0.500172,0.500170,0.500043,0.499836,0.500153,0.499696,0.500086,...,0.500153,0.50011,0.500098,0.50011,0.500120,0.499908,0.500172,0.500043,0.50011,0.499666
min,12347.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,13873.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,15320.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,0.000000,0.00000,0.000000,1.00000,1.000000,0.000000,1.000000,1.000000,0.00000,0.000000
75%,16789.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000
max,18283.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [15]:
dataWide = my_df.pivot_table(values='item_qty', index='customer_id', columns='item_id', fill_value=0)
dataWide.head()

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
customer_id,,,,,,,,,,,,,,,,,,,,,
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,15.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
12352.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,3.0,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
12353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Get 2nd Row(Customer id data)
dataWide[dataWide.index==12348.0]

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
customer_id,,,,,,,,,,,,,,,,,,,,,
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0


In [17]:
dataWide.iloc[1:2,:]

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
customer_id,,,,,,,,,,,,,,,,,,,,,
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0


In [18]:
dataWide.iloc[1,:]

item_id
20724      0.0
20725      0.0
20726      0.0
20727      0.0
20728      0.0
20914      0.0
21080      0.0
21212      0.0
21232      0.0
21754      0.0
21790      0.0
21931      0.0
21977     56.0
22178      0.0
22197      0.0
22355      0.0
84879      0.0
84991     56.0
85099B     0.0
85099C     0.0
85123A     0.0
Name: 12348.0, dtype: float64

### Item Based Colloborative Filtering

In [19]:
#Drop the Person column
data_ib = dataWide.copy()

In [20]:
data_ib.head()

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
customer_id,,,,,,,,,,,,,,,,,,,,,
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,15.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
12352.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,3.0,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
12353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
data_ib = data_ib.reset_index()

In [22]:
data_ib.head()

item_id,customer_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
0,12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0
2,12349.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,15.666667,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
3,12352.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
#Drop the customer_id column
#data_ib = data_ib.iloc[:,1:]
data_ib = data_ib.drop("customer_id", axis=1)

In [24]:
data_ib.head()

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,15.666667,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,3.0,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Create a placeholder dataframe listing item vs. item
data_ibs = pd.DataFrame(index=data_ib.columns,
                        columns=data_ib.columns)

In [26]:
data_ibs.head()

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
item_id,,,,,,,,,,,,,,,,,,,,,
20724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ## Similarity Measure 

In [27]:
for i in range(0,len(data_ibs.columns)) :
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)) :
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(data_ib.iloc[:,i],data_ib.iloc[:,j])

In [28]:
data_ibs.head()

item_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,21754,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
item_id,,,,,,,,,,,,,,,,,,,,,
20724,1,0.392925,0.306169,0.278711,0.314796,0.244653,0.221004,0.30429,0.164805,0.0949573,...,0.234539,0.174806,0.052682,0.011961,0.43836,0.0779143,0.176081,0.339787,0.30184,0.119243
20725,0.392925,1,0.333139,0.459197,0.562024,0.274353,0.440773,0.377806,0.180469,0.151744,...,0.342756,0.282124,0.0639042,0.0178983,0.356758,0.104326,0.319403,0.357454,0.263236,0.238774
20726,0.306169,0.333139,1,0.659696,0.678926,0.191686,0.175212,0.18361,0.107216,0.0555971,...,0.234525,0.110877,0.0329854,0.00981227,0.379977,0.0622654,0.135775,0.215558,0.208453,0.0938093
20727,0.278711,0.459197,0.659696,1,0.807393,0.142174,0.267916,0.204907,0.12132,0.0776594,...,0.342129,0.156281,0.0363166,0.0109616,0.236787,0.0658219,0.251643,0.24682,0.207655,0.186717
20728,0.314796,0.562024,0.678926,0.807393,1,0.175022,0.283132,0.22985,0.129636,0.104611,...,0.321453,0.177407,0.0289135,0.0114092,0.263943,0.0613303,0.236252,0.261864,0.204239,0.19447


In [29]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,5))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:4] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:4].index

In [30]:
data_neighbours

,1,2,3,4
item_id,,,,
20724,20724,22355,20725,85099B
20725,20725,20728,20727,21080
20726,20726,20728,20727,22355
20727,20727,20728,20726,20725
20728,20728,20727,20726,20725
20914,20914,21212,20725,21080
21080,21080,84991,21212,21977
21212,21212,84991,21977,21080
21232,21232,85123A,21931,85099B


In [31]:
#Exercise 4 Modify the above code to print the top 10 similar products for each product

In [32]:
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))
 
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:10].index
    
data_neighbours

,1,2,3,4,5,6,7,8,9,10
item_id,,,,,,,,,,
20724,20724,22355,20725,85099B,20728,20726,21212,85099C,20727,20914
20725,20725,20728,20727,21080,20724,21212,85099B,22355,21931,20726
20726,20726,20728,20727,22355,20725,20724,21931,85099B,85099C,20914
20727,20727,20728,20726,20725,21931,20724,21080,84991,85099B,22355
20728,20728,20727,20726,20725,21931,20724,21080,22355,85099B,84991
20914,20914,21212,20725,21080,20724,85099B,84991,85099C,20726,22355
21080,21080,84991,21212,21977,20725,20728,20727,20914,85099B,20724
21212,21212,84991,21977,21080,20725,20724,20914,20728,85099B,22355
21232,21232,85123A,21931,85099B,20725,85099C,20914,84991,21080,20724


### User Based collaborative Filtering

In [33]:
#Helper function to get similarity scores
def getScore(history, similarities):
   return sum(history*similarities)/sum(similarities)

In [34]:
data_sims1 = dataWide.reset_index()

In [35]:
data_sims1.head()

item_id,customer_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
0,12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,56.0,0.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0
2,12349.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,15.666667,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
3,12352.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data_sims1.index,columns=data_sims1.columns)
data_sims.iloc[:,:1] = data_sims1.iloc[:,:1]

In [37]:
#This is the same as our original data but with nothing filled in except the headers
data_sims.head()

item_id,customer_id,20724,20725,20726,20727,20728,20914,21080,21212,21232,...,21931,21977,22178,22197,22355,84879,84991,85099B,85099C,85123A
0,12347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
data_sims12 = data_sims1.iloc[:500,:]

In [39]:
data_sims11 = data_sims.iloc[:500,:]

In [40]:
product='20724'

In [41]:
product

'20724'

In [42]:
product_top_names = data_neighbours.loc[product][1:10]

In [43]:
product_top_names

2      22355
3      20725
4     85099B
5      20728
6      20726
7      21212
8     85099C
9      20727
10     20914
Name: 20724, dtype: object

In [44]:
#Need to run this for only 500 users. Might be slow beyond that. 
#Subset it to 500 users before running this
for i in range(0,len(data_sims11.index)):
    for j in range(1,len(data_sims11.columns)):
        user = data_sims11.index[i]
        product = data_sims11.columns[j]
 
        if data_sims12.iloc[i][j] == 1:
            data_sims11.iloc[i][j] = 0
        else:
            product_top_names = data_neighbours.loc[product][1:10]
            product_top_sims = data_ibs.loc[product].sort_values(ascending=False)[1:10]
            user_purchases = data_ib.loc[user,product_top_names]
            
            print (i)
            print (j)
 
            data_sims11.iloc[i][j] = getScore(user_purchases,product_top_sims)

0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
0
15
0
16
0
17
0
18
0
19
0
20
0
21
1
1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
1
15
1
16
1
17
1
18
1
19
1
20
1
21
2
1
2
2
2
3
2
4
2
5
2
6
2
7
2
8
2
9
2
10
2
11
2
12
2
13
2
14
2
15
2
16
2
17
2
18
2
19
2
20
2
21
3
1
3
2
3
3
3
4
3
5
3
6
3
7
3
8
3
9
3
10
3
11
3
12
3
13
3
14
3
15
3
16
3
17
3
18
3
19
3
20
3
21
4
1
4
2
4
3
4
4
4
5
4
6
4
7
4
8
4
9
4
10
4
11
4
12
4
13
4
14
4
15
4
16
4
17
4
18
4
19
4
20
4
21
5
1
5
2
5
3
5
4
5
5
5
6
5
7
5
8
5
9
5
10
5
11
5
12
5
13
5
14
5
15
5
16
5
17
5
18
5
19
5
20
5
21
6
1
6
2
6
3
6
4
6
5
6
6
6
7
6
8
6
9
6
10
6
11
6
12
6
13
6
14
6
15
6
16
6
17
6
18
6
19
6
20
6
21
7
1
7
2
7
3
7
4
7
5
7
6
7
7
7
8
7
9
7
10
7
11
7
12
7
13
7
14
7
15
7
16
7
17
7
18
7
19
7
20
7
21
8
1
8
2
8
3
8
4
8
5
8
6
8
7
8
8
8
9
8
10
8
11
8
12
8
13
8
14
8
15
8
16
8
17
8
18
8
19
8
20
8
21
9
1
9
2
9
3
9
4
9
5
9
6
9
7
9
8
9
9
9
10
9
11
9
12
9
13
9
14
9
15
9
16
9
17
9
18
9
19
9
20
9
21
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
8


In [55]:
# Get the top products
data_recommend = pd.DataFrame(index=data_sims.index, columns=['customer_id','TopProduct_1','TopProduct_2','TopProduct_3','TopProduct_4','TopProduct_5'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]

In [56]:
data_recommend

,customer_id,TopProduct_1,TopProduct_2,TopProduct_3,TopProduct_4,TopProduct_5
0,12347,NaN,NaN,NaN,NaN,NaN
1,12348,NaN,NaN,NaN,NaN,NaN
2,12349,NaN,NaN,NaN,NaN,NaN
3,12352,NaN,NaN,NaN,NaN,NaN
4,12353,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3875,18280,NaN,NaN,NaN,NaN,NaN
3876,18283,NaN,NaN,NaN,NaN,NaN
3877,18285,NaN,NaN,NaN,NaN,NaN
3878,18286,NaN,NaN,NaN,NaN,NaN


In [57]:
# Instead of top product scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending=False).iloc[1:6,].index.transpose()

In [58]:
data_recommend

,customer_id,TopProduct_1,TopProduct_2,TopProduct_3,TopProduct_4,TopProduct_5
0,12347,21977,21212,21080,20914,21232
1,12348,21212,21080,21977,21790,84991
2,12349,22197,21754,22178,85123A,20914
3,12352,84879,85123A,22178,21754,21790
4,12353,85123A,21754,22178,21212,20724
...,...,...,...,...,...,...
3875,18280,20724,20725,20726,20727,20728
3876,18283,20724,20725,20726,20727,20728
3877,18285,20724,20725,20726,20727,20728
3878,18286,20724,20725,20726,20727,20728


In [61]:
# Print a sample
data_recommend=data_recommend.iloc[:,:4]

In [62]:
data_recommend.to_csv (r'Recommended_Products.csv', index = False, header=True)